In [6]:
from spytial import *
from spytial.annotations import *

In [7]:
# Setup for performance metrics
import random
perf_base = "spytial_perf"
def get_perf_path(structure, size):
    return perf_base + "_" + structure + "_" + f"{size}.json"
PI = 20


SIZES = [3, 4, 7, 10]  # Corresponding chain lengths


def get_target_cells(size):
    """
    Calculate the number of cells in the upper triangular matrix (including diagonal)
    for a matrix chain of given size.
    
    For a chain of n matrices, the m and s tables are (n+1) x (n+1), but only the 
    upper triangle is filled (where j >= i). This is a triangular number: n*(n+1)/2
    
    Args:
        size: The chain length (number of matrices)
    
    Returns:
        Number of cells that will be computed/displayed in the triangular matrix
    """
    return size * (size + 1) // 2
TARGETS = {size: get_target_cells(size) for size in SIZES}
print("Target cells per size:", TARGETS)

Target cells per size: {3: 6, 4: 10, 7: 28, 10: 55}


# Memoization Data Structure 
m and s tables computed by Matrix-Chain-Order

**Have to wrap, since matrix chains in diff order**

In [8]:
# Memoization Data Structure
from math import inf
from functools import lru_cache

def memoized_matrix_chain(p):
    """
    p: dims vector of length n+1 (A1 is p[0]x p[1], ..., An is p[n-1]x p[n])
    returns: m, s where m[i][j] = min cost, s[i][j] = optimal split k
    1-indexed to match CLRS.
    """
    n = len(p) - 1
    m = [[None]*(n+1) for _ in range(n+1)]
    s = [[None]*(n+1) for _ in range(n+1)]

    @lru_cache(None)
    def lookup(i, j):
        if i == j:
            m[i][j] = 0
            return 0
        best, best_k = inf, None
        for k in range(i, j):
            left = lookup(i, k)
            right = lookup(k+1, j)
            cost = left + right + p[i-1]*p[k]*p[j]
            if cost < best:
                best, best_k = cost, k
        m[i][j] = best
        s[i][j] = best_k
        return best

    lookup(1, n)
    return m, s



##CLRS (Cormen, Leiserson, Rivest, Stein - Introduction to Algorithms, Chapter 15) uses the bottom-up approach. They describe the algorithm with nested loops that fill the tables iteratively, exactly like your bottom_up_matrix_chain function. This is the standard presentation in the textbook, as it aligns with how they explain dynamic programming tables.
def bottom_up_matrix_chain(p):
    n = len(p)-1
    m = [[0 if i==j else inf for j in range(n+1)] for i in range(n+1)]
    s = [[None]*(n+1) for _ in range(n+1)]
    for L in range(2, n+1):             # chain length
        for i in range(1, n-L+2):
            j = i+L-1
            m[i][j] = inf
            for k in range(i, j):
                q = m[i][k] + m[k+1][j] + p[i-1]*p[k]*p[j]
                if q < m[i][j]:
                    m[i][j] = q
                    s[i][j] = k
    return m, s

def print_tables(m, s):
    # compact CLRS-style triangular display
    n = len(m)-1
    def row_fmt(row): return " ".join(("{:>6}".format("" if x is None or x==float('inf') else x) for x in row[1:]))
    print("m-table:")
    for i in range(1, n+1):
        row = [None]*(i) + m[i][i:]  # leading blanks
        print(row_fmt(row))
    print("\ns-table (k splits):")
    for i in range(1, n+1):
        row = [None]*(i) + s[i][i:]
        print(row_fmt(row))

def parenthesize(s, i, j):
    if i == j: return f"A{i}"
    k = s[i][j]
    return f"({parenthesize(s, i, k)}{parenthesize(s, k+1, j)})"


![memo](img/memoization.png)

In [9]:

# Example (CLRS Fig. 15.5 uses p = [30,35,15,5,10,20,25])
p = [30,35,15,5,10,20,25]
m, s = bottom_up_matrix_chain(p)   # or memoized_matrix_chain(p)


In [10]:
# Create a wrapper class to make matrix cells explicit atoms
@attribute(field='value')
@attribute(field='row')
@attribute(field='col')
class MatrixCell:
    def __init__(self, row, col, value):
        self.row = row
        self.col = col
        self.value = value


SAME_ROW = "(row.~row)"
SAME_COL = "(col.~col)"

DIFF_ROWS = "row.{i,j : int | i < j}.~row"
DIFF_COLS = "col.{i,j : int | i < j}.~col"


@atomColor(selector='{m : MatrixCell | (m.row) = 0 and (m.col) = 0}', value = 'black')
@hideAtom(selector='MatrixWrapper + list + int')
@align(selector=SAME_ROW, direction='horizontal')
@align(selector=SAME_COL, direction='vertical')
@orientation(selector=DIFF_ROWS, directions=['below'])
@orientation(selector=DIFF_COLS, directions=['right'])
class MatrixWrapper:
    def __init__(self, matrix, skip_inf=True):
        self.cells = []
        for i, row in enumerate(matrix):
            if row is None:
                continue
            for j, val in enumerate(row):
                if val is None or (skip_inf and val == inf):
                    continue
                self.cells.append(MatrixCell(i, j, val))

# Usage:
p = [30,35,15,5,10,20,25]
m, s = bottom_up_matrix_chain(p)
m_wrapped = MatrixWrapper(m, skip_inf=True)
diagram(m_wrapped)


## Performance

In [11]:
STRUCTURE = "memoization_matrix"
for size in SIZES:
    # Generate random dimension vector (size+1 dimensions for size matrices)
    p = [random.randint(5, 50) for _ in range(size)]
    
    m, s = bottom_up_matrix_chain(p)
    m_wrapped = MatrixWrapper(m, skip_inf=True)
    cell_count = get_target_cells(size)
    diagram(m_wrapped, method="browser", perf_path=get_perf_path(STRUCTURE, cell_count), perf_iterations=PI, title=f"Matrix Chain  (Size={cell_count})", headless=True, timeout=600)

Building data instance for: Matrix Chain  (Size=6)
Generating HTML for: Matrix Chain  (Size=6)
Loading page for Matrix Chain  (Size=6) (timeout: 600s)...
Running 20 iterations - Matrix Chain  (Size=6) (timeout: 600s)...
  Using custom timeout of 600s
  Progress: 20/20 iterations (10.0s elapsed)
✓ Headless benchmark completed: 20 iterations
  Generate Layout: 11.42ms avg
  Render Layout: 15.45ms avg
  Total Time: 27.37ms avg
  Metrics saved to: spytial_perf_memoization_matrix_6.json
Building data instance for: Matrix Chain  (Size=10)
Generating HTML for: Matrix Chain  (Size=10)
Loading page for Matrix Chain  (Size=10) (timeout: 600s)...
Running 20 iterations - Matrix Chain  (Size=10) (timeout: 600s)...
  Using custom timeout of 600s
  Progress: 20/20 iterations (10.0s elapsed)
✓ Headless benchmark completed: 20 iterations
  Generate Layout: 12.08ms avg
  Render Layout: 106.39ms avg
  Total Time: 118.95ms avg
  Metrics saved to: spytial_perf_memoization_matrix_10.json
Building data insta